In [1]:
from Crypto.PublicKey import RSA
import MCipher

In [2]:
prikey = RSA.generate(1024)
pubkey = prikey.publickey()

priFile = open("./clientrpriKey.pem" , "wb+")
priFile.write(prikey.exportKey("PEM"))
priFile.close()

pubFile = open("./clientpubKey.pem" , "wb+")
pubFile.write(pubkey.exportKey("PEM"))
pubFile.close()

In [3]:
import socket
import MCipher 

def client_program() :
    host = '168.188.126.25'
    port = 5461
    
    keyReceive = False
    client_socket = socket.socket()
    client_socket.connect((host, port))

    if keyReceive == False :
        key_recv = client_socket.recv(1024)
        
        client_prikey = MCipher.readPEM("clientrpriKey.pem")
        key = MCipher.RSADecrypt(client_prikey, key_recv)
        print("key : " , key)
        
        client_socket.send('key exchange Success'.encode())
        
        iv = client_socket.recv(1024).decode()
        print('iv  :' , iv)
        client_socket.send('iv exchange Success'.encode())
        
        keyReceive = True
        
    AES = MCipher.setAES(key, iv)
    
    if keyReceive :
        message = input(" -> ")
        
        while message.lower().strip() != 'bye' :
            
            
            hash_block = MCipher.makeHashBlock(message)   # mesage hash
            hash_block_en = MCipher.AES_Encrypt(AES,hash_block)
            client_socket.send(hash_block_en)
        
        
        
            data = client_socket.recv(1024)

            d_data = MCipher.AES_Decrypt(AES,data)
            data, hash_data = MCipher.separateHashBlcok(d_data)
            if MCipher.integrityCheck(data , hash_data) :
                print("무결성 만족")
                print("revc complete.....")
                print("Recieved from user2 :" + str(data))

                data = input('->')

            else :
                print("무결성 위배")
                break
    
    client_socket.close()
        
    
if __name__ == '__main__' :
    client_program()

TimeoutError: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다